In [ ]:
import pandas as pd
import numpy as np
import os
import gc
from tqdm import tqdm_notebook # check the progressbar in the python. 
import glob # check the file name in fold. 

In [ ]:
import pyspark

import findspark
findspark.init()
findspark.find()

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
path = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/'

In [ ]:
read_file_lst = glob.glob(path + '*')
read_file_lst[0:2]

In [ ]:
from pyspark import SparkContext
sc = SparkContext()

In [ ]:
import functools 

def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkFiles
from pyspark.sql.functions import lit

sqlContext = SQLContext(sc)
for i in tqdm_notebook(read_file_lst):
    # sqlContext.read.csv() : path를 C:Users/..의 형식으로 맞춰줘야함. (초기값이 spark설치된 곳으로 지정되어있음)
    df_temp = sqlContext.read.csv(SparkFiles.get(i), header=True, inferSchema= True) # read.csv 
    
    # transformation으로 추가하려면, df = df.withColumn("age_square", col("age")**2)
    df_temp = df_temp.withColumn("date", lit(i[-8:-4])) # column 추가, lit명령어가 value를 추가하는 방법. 
    
    if i == read_file_lst[0]:
        df_total = df_temp
    else:
        df_total = unionAll([df_total, df_temp]) # row-wise 결합
        
    del df_temp 
    gc.collect()

In [ ]:
import pyspark.sql.functions as f
# userItem=df.groupby('userId').agg(f.expr('count(distinct item)').alias('n_item'))
# df_total.groupBy("log_id").agg(f.expr('count(distinct item)').alias('log_id'))
# df_total.select("actor_account").distinct().count()

In [ ]:
i = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/'
train_label = pd.read_csv(i + "labeled_accounts.csv")
test_label = pd.read_csv(i + "test_accounts.csv")
del train_label['class']

### Feature Engineering

Player information features
- Actor (o)
- A_Acc (o)
- login_day_count 
- logout_day_count
- playtime (o)
- playtime_per_day
- avg_money (o)
- login_count (o)
- ip_count (o)
- max_level (o)

Feature name : login_count

In [ ]:
import pyspark.sql.functions as func # pyspark의 유용한 기능을 사용하는 패키지 : countDistinct 사용가능
# 원하는 부분만 출력할때는 df.filter()를 사용. 

# 아이온데이터에 actor는 다르지만 actor_account가 같은 경우는 있는 것 같음. 
# 하지만 제출형태가 actor_account를 기준으로 해가지고 groupby를 아래의 변수로 진행했음. 
df_total = df_total.withColumnRenamed("actor_account", "account")

df_total_agg = df_total.filter(df_total['log_id'] == 103).select('account').toPandas()
df_total_agg = df_total_agg.groupby('account')['account'].agg({'count'}).reset_index().rename(columns={'count':'login_count'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

Feature name : Day_unique_count 

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 103).select('account','date').toPandas()
df_total_agg = df_total_agg.groupby('account')['date'].agg({'nunique'}).reset_index().rename(columns={'nunique':'Day_unique_count'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

Feature name : ip_count

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 103).select('account','etc_str1').toPandas()
df_total_agg = df_total_agg.groupby('account')['etc_str1'].agg({'nunique'}).reset_index().rename(columns={'nunique':'ip_nunique'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

Feature name : max_level

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 103).select('account','etc_num2').toPandas()
df_total_agg = df_total_agg.groupby('account')['etc_num2'].agg({'max'}).reset_index().rename(columns={'max':'max_level'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

Feature name : playtime

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 104).select('account','etc_num7').toPandas()
df_total_agg = df_total_agg.groupby('account')['etc_num7'].agg({'mean', 'sum'}).reset_index().rename(columns={'mean':'mean_playtime', 'sum':'sum_playtime'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

Feature name : sum_money

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 104).select('account','etc_num1').toPandas()
df_total_agg = df_total_agg.groupby('account')['etc_num1'].agg({'mean', 'sum'}).reset_index().rename(columns={'sum':'sum_money'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 116).select('account','etc_num7','etc_num8','etc_num9','etc_num10').toPandas()
df_total_agg = df_total_agg.groupby('account')[['etc_num7','etc_num8','etc_num9','etc_num10']].agg({'mean'}).reset_index()
df_total_agg.columns = ['account', 'captcha_count_mean', 'captcha_ban_time_mean', 'captcha_prior_time_mean', 'captcha_occur_time_mean']

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
outputpath = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/features/'
train_label.to_csv(outputpath + "train_player.csv", index=False)
test_label.to_csv(outputpath + "test_player.csv", index=False)